In [17]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from typing import Any
from google.oauth2 import service_account
import logging
from datetime import datetime, timedelta

credentials = service_account.Credentials.from_service_account_file("connection-123-892e002c2def.json")
destination_table = "btc_ema"

logger = logging.getLogger(__name__)

def calculate_ema(credentials,periods=[12, 26, 20, 50, 200]) -> pd.DataFrame:
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    query = """
    SELECT 
        timestamp, 
        price 
    FROM 
        `connection-123.signals.bitcoin_price` 
    WHERE 
        DATE(timestamp) < DATE(CURRENT_TIMESTAMP())
    ORDER BY 
        timestamp ASC
    """
    
    query_job = client.query(query)
    results = query_job.result()
    ema = results.to_dataframe()
    bytes_processed = query_job.total_bytes_processed
    logger.info(f"Query processed {bytes_processed:,} bytes ({bytes_processed / 1024 / 1024:.2f} MB)")

    # Calculate EMA for each period
    for period in periods:
        ema_column = f'ema_{period}'
        # Calculate EMA using the 'price' column
        ema[ema_column] = ema['price'].ewm(span=period, adjust=False).mean()
    
    return ema


In [18]:
data = calculate_ema(credentials)

In [19]:
print(data)

      timestamp          price         ema_12         ema_26         ema_20  \
0    2024-08-05   58006.206587   58006.206587   58006.206587   58006.206587   
1    2024-08-06   53956.261842   57383.138164   57706.210680   57620.497563   
2    2024-08-07   55959.841074   57164.169381   57576.849968   57462.339802   
3    2024-08-08   55099.951811   56846.597447   57393.376031   57237.350470   
4    2024-08-09   61859.031599   57617.741163   57724.165332   57677.510578   
..          ...            ...            ...            ...            ...   
361  2025-08-01  115700.002439  117560.961975  116026.238229  116815.269357   
362  2025-08-02  113234.605134  116895.368615  115819.450592  116474.253717   
363  2025-08-03  112554.902322  116227.604570  115577.632202  116100.982156   
364  2025-08-04  114199.109665  115915.528431  115475.519421  115919.851442   
365  2025-08-04  114779.262059  115740.718220  115423.944802  115811.223882   

            ema_50        ema_200  
0     58006.206